In [1]:
import pandas as pd
import regex as re

import emoji

from tqdm import tqdm
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

In [2]:
tqdm.pandas()

In [3]:
# file_path = 'train-00000-of-00041-3f49db2da17edd5a.parquet'
# file_path = 'train-00001-of-00041-4491d6daeecba187.parquet'

file_path = 'train-00001-of-00041-4491d6daeecba187.parquet'
df = pd.read_parquet(file_path)
df.head(10)

,user,id,tweet,replies,retweets,likes,quotes,date
0,GlobalDubbing,919215935575248897,TESTERE : JIGSAW EFSANESİ // PERFORMANS // TUĞ...,0,0,1,0,2017-10-14T14:58:51+00:00
1,GlobalDubbing,919139242680487936,TESTERE : JIGSAW EFSANESİ // PERFORMANS // BUR...,0,0,8,0,2017-10-14T09:54:06+00:00
2,GlobalDubbing,919134110504968192,TESTERE : JIGSAW EFSANESİ // PERFORMANS // ELİ...,0,4,15,0,2017-10-14T09:33:43+00:00
3,GlobalDubbing,918525666869596160,TESTERE : JIGSAW EFSANESİ // PERFORMANS // SİN...,0,0,3,0,2017-10-12T17:15:59+00:00
4,GlobalDubbing,918018472004767744,TESTERE : JIGSAW EFSANESİ // PERFORMANS // ALİ...,0,0,0,0,2017-10-11T07:40:34+00:00
5,GlobalDubbing,917031204628652034,DEATH WISH SESLENDİRME KADROSU🎙 via @SesKadrol...,0,5,6,0,2017-10-08T14:17:31+00:00
6,GlobalDubbing,917030990421417986,TESTERE : JIGSAW EFSANESİ SESLENDİRME KADROSU🎙...,0,0,3,0,2017-10-08T14:16:40+00:00
7,GlobalDubbing,916649378894499840,DEATH WISH // SESLENDİRME KADROSU // HAKAN VAN...,0,3,10,0,2017-10-07T13:00:17+00:00
8,GlobalDubbing,916640628993380353,DEATH WISH // SESLENDİRME KADROSU // MERİH TAN...,0,0,3,0,2017-10-07T12:25:30+00:00
9,GlobalDubbing,916635955339161600,DEATH WISH // SESLENDİRME KADROSU // ÖMER ATAL...,0,0,1,0,2017-10-07T12:06:56+00:00


In [4]:
df.shape

(2148399, 8)

In [5]:
emojis_to_search = [
    "👍",
    "🙏",
    "🎉",
    "🔥",
    "🤦",
    "❤️",
    "🤷",
    "🎂",
    "👏",
    "👌",
    "💪",
    "✨",
    "👀",
    "👉",
    "🌹",
    "🎈",
    "💐",
    "🤞",
    "🙌",
    "👇",
    "🌞",
    "🌸",
    "🎶",
    "✌️",
    "🎊",
    "☀️",
    "💰",
    "👑",
    "🎁",
    "🙋"]

In [6]:
def contain_emoji(text):
    emoji_pattern = re.compile('|'.join(re.escape(emoji) for emoji in emojis_to_search))
    return bool(emoji_pattern.search(text))

def clean_tags(text):
    pattern = r'@\S+\b | #\S+\b'
    text = re.sub(pattern, '', text)
    pattern = r'https://\S+\b'
    return re.sub(pattern, 'URL', text)

def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except LangDetectException:
        return False

df['contain_emoji'] = df['tweet'].progress_apply(contain_emoji)
df = df[df['contain_emoji']]

df['tweet'] = df['tweet'].progress_apply(clean_tags)

df['is_english'] = df['tweet'].progress_apply(is_english)
df = df[df['is_english']]

100%|█████████████████████████████████████████████████████████████████████████| 176647/176647 [10:16<00:00, 286.59it/s]


In [7]:
df[['tweet']].reset_index(drop=True).head(10)

,tweet
0,Merry Christmas and Happy Holidays from all of...
1,Our Tradefora Team has made it to the Final Ro...
2,"2018 was an amazing year for us, more to come ..."
3,Its official - after years of R&amp;D and hard...
4,Yay for the win!!! My new fave tv show ... So ...
5,Birthday breakfast balthazarrestaurant 🎉🎉🎉 w m...
6,#reunited with after two years and it feels so...
7,Saturday salt water pool vibes as I memorize s...
8,❤️👓🎶💞🙏 all day everyday @ The Magic Shop Recor...
9,No matter how crazy life gets ... Never forget...


In [8]:
def extract_emojis(text):
    return ''.join(_['emoji'] for _ in emoji.emoji_list(text))

def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

tweets = df['tweet'].tolist()

texts = [remove_emojis(tweet) for tweet in tweets]

emojis = [extract_emojis(tweet) for tweet in tweets]

In [9]:
clean_df = pd.DataFrame({'text': texts, 
                         'emoji': emojis})

for em in emojis_to_search:
    clean_df[em] = clean_df['emoji'].apply(lambda x: em in x)

clean_df.head(10)

,text,emoji,👍,🙏,🎉,🔥,🤦,❤️,🤷,🎂,...,🌞,🌸,🎶,✌️,🎊,☀️,💰,👑,🎁,🙋
0,Merry Christmas and Happy Holidays from all of...,🌲🎊🌲,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,Our Tradefora Team has made it to the Final Ro...,🎊🍀🍀🍀,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,"2018 was an amazing year for us, more to come ...",🎊🎆🎉🎈🎄,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,Its official - after years of R&amp;D and hard...,🎊🎈🎊,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,Yay for the win!!! My new fave tv show ... So ...,👏🏻👏🏻👏🏻,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,Birthday breakfast balthazarrestaurant w my o...,🎉🎉🎉,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,#reunited with after two years and it feels so...,❤️🎶,False,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
7,Saturday salt water pool vibes as I memorize s...,🙆😘🙏💦🔪💉💰,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
8,all day everyday @ The Magic Shop Recording S...,❤️👓🎶💞🙏,False,True,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
9,No matter how crazy life gets ... Never forget...,💐🌺🌸💐💞💞💞,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [ ]:
clean_df.to_csv(file_path[:11] + '.csv', index = False)  